<a href="https://colab.research.google.com/github/andrewli4938/TrafficLightDetection/blob/main/Group1_Traffic_Light_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 879.0/879.0 kB 16.2 MB/s eta 0:00:00


In [2]:
import torch
import torch.nn as nn
import pandas as pd
from sklearn.model_selection import train_test_split
import os
import glob
import shutil
from PIL import Image
from google.colab import drive

from ultralytics import YOLO

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [ ]:
# import our dataset
import kagglehub
source_path = kagglehub.dataset_download("mbornoe/lisa-traffic-light-dataset")

100%|██████████| 4.21G/4.21G [00:54<00:00, 83.2MB/s]

Extracting files...


In [ ]:
%pwd
%rm -r data
%mkdir data  # root folder for data

%cd data
%mkdir images
%mkdir labels

%cd images
%mkdir train
%mkdir val
%mkdir test

%cd ../labels
%mkdir train
%mkdir val
%mkdir test

%cd /content

In [ ]:
# these two only have total 1000 images, and have duplicate filenames so we can deal with them later
%mv {source_path}/sample-dayClip6 {source_path}/..
%mv {source_path}/sample-nightClip1 {source_path}/..

In [ ]:
# move all of the filenames into list
image_paths = glob.glob(f"{source_path}/**/*.jpg", recursive=True)

# process the annotations
target = "frameAnnotationsBOX.csv"
annotations = pd.DataFrame()
annotation_paths = glob.glob(f"{source_path}/**/{target}", recursive=True)
for p in annotation_paths:
  # combine the csv files into one
  new_frame = pd.read_csv(p, sep=";")
  new_frame = new_frame.drop(['Origin file', 'Origin frame number', 'Origin track', 'Origin track frame number'], axis=1)
  annotations = pd.concat([annotations, new_frame])
filenames = annotations[annotations.columns[0]].str.split("/").str[-1]
annotations["Filename"] = filenames
annotations.set_index("Filename", inplace=True)
annotations

In [ ]:
def move_files(paths_list, destination_dir):
  for file in paths_list:
    shutil.copy(file, destination_dir)

In [ ]:
# TODO: split the data into train, val, test
image_train, image_val_test = train_test_split(image_paths, train_size=0.7, test_size=0.3, random_state=42, shuffle=True)
image_val, image_test = train_test_split(image_val_test, train_size=0.5, random_state=42, shuffle=True)

print(len(image_train))
print(len(image_val))
print(len(image_test))

image_train_dir = "/content/data/images/train"
image_val_dir = "/content/data/images/val"
image_test_dir = "/content/data/images/test"
move_files(image_train, image_train_dir)
move_files(image_val, image_val_dir)
move_files(image_test, image_test_dir)

print(len(os.listdir(image_train_dir)))
print(len(os.listdir(image_val_dir)))
print(len(os.listdir(image_test_dir)))

In [ ]:
# import yaml file: defines image locations and encodes classes into numbers for the model
!wget https://raw.githubusercontent.com/andrewli4938/TrafficLightDetection/refs/heads/main/data.yaml

In [ ]:
def convert_to_xywh(box_coordinates, image_dimensions):
  image_width, image_height = image_dimensions
  x1, y1, x2, y2 = box_coordinates
  width, height = x2-x1, y2-y1
  x_center = (x1+x2)/2
  y_center = (y1+y2)/2

  x_center = x_center/image_width
  y_center = y_center/image_height
  width = width/image_width
  height = height/image_height

  return (x_center, y_center, width, height)

In [ ]:
"""
The *.txt file should be formatted with one row per object in
class x_center y_center width height format. Box coordinates
must be in normalized xywh format (from 0 to 1). If your boxes
are in pixels, you should divide x_center and width by image width,
and y_center and height by image height. Class numbers should be
zero-indexed (start with 0).
"""
image_dims = (1280, 960)

def create_labels(source_dir, destination_dir, annotations_df, encoding):
  bounding_box_df = annotations_df.iloc[:, 1:]
  for file in os.listdir(source_dir):
    write_path = os.path.join(destination_dir, file.replace(".jpg", ".txt"))
    if file in annotations_df.index:
      rows = annotations_df.loc[file]
      if isinstance(rows, pd.Series):
        rows = pd.DataFrame([rows])
    else:
      continue
    with open(write_path, "w") as image_label:
      for i in range(len(rows)):  # iterate through each label for current image
        class_number = encoding[rows.iloc[i, 0]]
        bounding_box = bounding_box_df.iloc[i]
        # print(bounding_box)
        # print(f"image is: {file} at index {i}")
        # print(type(rows))
        xywh = convert_to_xywh(bounding_box, image_dims)  # calculate bounding box coordinates here
        image_label.write(f"{class_number} {xywh[0]} {xywh[1]} {xywh[2]} {xywh[3]}\n")


In [ ]:
class_encoding = {"stop": 0, "stopLeft": 1, "warning": 2, "warningLeft": 3,
                  "warningLeft": 4, "go": 5, "goForward": 6, "goLeft": 6}

label_train_dir = "/content/data/labels/train"
label_val_dir = "/content/data/labels/val"
label_test_dir = "/content/data/labels/test"

create_labels(image_train_dir, label_train_dir, annotations, class_encoding)
create_labels(image_val_dir, label_val_dir, annotations, class_encoding)
create_labels(image_test_dir, label_test_dir, annotations, class_encoding)

print(len(os.listdir(label_train_dir)))
print(len(os.listdir(label_val_dir)))
print(len(os.listdir(label_test_dir)))

In [ ]:
torch.cuda.is_available()

In [ ]:
model = YOLO("yolo11n.pt")
results = model.train(data="/content/data.yaml", epochs=10, imgsz=512, batch=4)

In [25]:
annotations.loc["nightSequence2--06532.jpg"]

,Annotation tag,Upper left corner X,Upper left corner Y,Lower right corner X,Lower right corner Y
Filename,,,,,
nightSequence2--06532.jpg,go,673,329,683,344
nightSequence2--06532.jpg,go,709,158,739,209
nightSequence2--06532.jpg,go,962,252,994,312
nightSequence2--06532.jpg,go,716,350,723,362
nightSequence2--06532.jpg,stop,651,333,657,343
nightSequence2--06532.jpg,stopLeft,563,168,573,178
nightSequence2--06532.jpg,stopLeft,651,333,657,343


In [23]:
!cat /content/data/labels/val/nightSequence2--06532.txt

5 0.524609375 0.4088541666666667 0.00703125 0.015625
5 0.4109375 0.4140625 0.009375 0.021875
5 0.572265625 0.41510416666666666 0.01171875 0.028125
5 0.524609375 0.4088541666666667 0.00703125 0.015625
0 0.4109375 0.41302083333333334 0.009375 0.021875
1 0.572265625 0.4140625 0.01171875 0.028125
1 0.524609375 0.40677083333333336 0.00703125 0.015625


In [ ]:
#TODO: the label files are producing duplicate files, fix the create_labels() function

In [ ]:
"""
OVERVIEW
Basically our LISA dataset contains around 44k images and a bunch of labels

Labels are:
  - light status: {go, slow, stop}
  - bounding box of where the stoplight is xyxy coordinates (top left, bottom right)
  - each image may have many labels since there are many stoplights in a single image

DaySequence
"""

In [16]:
"""
END HERE
"""

'\nEND HERE\n'

In [11]:
# # centralize both annotations and images (not scattered in different folders)
# annotations = pd.DataFrame()

# target = "frameAnnotationsBOX.csv"
# annotation_paths = glob.glob(f"{path}/**/{target}", recursive=True)

# # I am going to omit the approximately 1000 pictures and labels in sample-dayClip6
# # and sample-nightClip1 because it is really annoying to work with right now and and
# # we have 43k other values we can work with first
# for p in annotation_paths:
#   tokens = p.split("/")
#   if tokens[-2] == "sample-nightClip1" or tokens[-2] == "sample-dayClip6":  # omit here
#     continue

#   # combine the csv files into one
#   new_frame = pd.read_csv(p, sep=";")
#   new_frame = new_frame.drop(['Origin file', 'Origin frame number', 'Origin track', 'Origin track frame number'], axis=1)
#   annotations = pd.concat([annotations, new_frame])

#   # move all of the nested image files into /content/images_dir
#   if len(tokens)==14:
#     subpath = path+"/"+tokens[-3]+"/"+tokens[-3]+"/"+tokens[-2]+"/frames"
#   else:
#     subpath = path+"/"+tokens[-2]+"/"+tokens[-2]+"/frames"
#   !cp -r {subpath}/* /content/images_dir

# filenames = annotations[annotations.columns[0]].str.split("/").str[-1]
# annotations["Filename"] = filenames
# annotations  # NOTE: our labels are in this df here

,Filename,Annotation tag,Upper left corner X,Upper left corner Y,Lower right corner X,Lower right corner Y
0,nightSequence2--00000.jpg,stop,566,335,572,348
1,nightSequence2--00000.jpg,stop,695,308,705,329
2,nightSequence2--00000.jpg,stop,745,335,752,345
3,nightSequence2--00000.jpg,stop,678,331,684,341
4,nightSequence2--00001.jpg,stop,564,333,571,346
...,...,...,...,...,...,...
18509,nightSequence1--04860.jpg,go,873,86,1018,206
18510,nightSequence1--04861.jpg,go,946,74,1091,188
18511,nightSequence1--04862.jpg,go,1016,51,1161,177
18512,nightSequence1--04863.jpg,go,1085,42,1245,159


In [33]:
# #TODO: define custom dataset class
# class TrafficLightDataset(Dataset):
#   def __init__(self, annotations_df, img_dir, transform=None, target_transform=None):
#     self.img_labels = annotations_df
#     self.img_dir = img_dir
#     self.transform = transform
#     self.target_transform = target_transform

#   def __len__(self):
#     return len(self.img_labels)

#   def __getitem__(self, idx):
#     img_path = os.path.join(self.img_dir, self.img_labels.iloc[idx, 0])
#     image = read_image(img_path)
#     label = tuple(self.img_labels.iloc[idx, 1:])
#     if self.transform:
#       image = self.transform(image)
#     if self.target_transform:
#       pass  # not sure how this will work on a tuple (worry about it later)
#     return image, label


In [34]:
# #TODO: split our data into train and test dataloaders

# img_dir = "content/images_dir"
# dataset = TrafficLightDataset(annotations, img_dir, transform=None, target_transform=None)

# # shuffle our indices before splitting (need day and night in both sets)
# indices = torch.randperm(len(dataset))
# train_size = int(0.8*len(dataset))
# train_indices = indices[:train_size]
# test_indices = indices[train_size:]

# # split into train and test
# train_dataset = torch.utils.data.Subset(dataset, train_indices)
# test_dataset = torch.utils.data.Subset(dataset, test_indices)

# # we have our dataloaders here
# train_dataloader = DataLoader(train_dataset, batch_size=64, shuffle=True)
# test_dataloader = DataLoader(test_dataset, batch_size=1000, shuffle=True)